**Trigger**

## Carlos Hurtado

Un trigger es un objeto almacenado en la base de datos que se ejecuta automaticamente en respuesta a la modificacion de los datos de una tabla. 

Estas modificaciones incluyen:

* insert 
* update 
* delete 

MariaDB soporta los siguientes tipos de triggers:

* Insert trigger: Se ejecuta automaticamente cuando una instruccion  insert agrega una nueva fila a una tabla. 

* Update trigger: Se activa automaticamente cuando una declaracion update modifica los datos de una tabla.

* Delete trigger: Se invoca automaticamente cuando una declaracion delete  elimina una o mas filas de una tabla.


MariaDB clasifica cada tipo de trigger por el tiempo del mismo como before y after:

* Before trigger: Se activa justo antes de que ocurra un evento 

* After trigger: Se activa justo despues de que ocurra un evento 

Para crear un nuevo trigger en la base de datos se utiliza la siguiente sintaxis:

create trigger trigger_name

{before | after} {insert | update | delete }

on table_name for each row

trigger_body;

En esta sintaxis:

* Primero se especifica el nombre del trigger que usted quiere crear.
* Segundo, se especifica el tiempo de accion: before o after.
* Tercero, se especifica el evento que activa el trigger: insert, update y delete.
* Cuarto, indique el nombre de la tabla a la que pertenece el trigger después de la palabra clave on. 
* Finalmente, especifique una declaración o declaraciones para ejecutar cuando se invoca el trigger.

Si desea ejecutar varias instrucciones, colóquelas dentro de la instrucción compuesta BEGIN END.

Dentro del cuerpo del trigger, puede acceder a los valores de las columnas que se ven afectadas por una instrucción: insert, update o delete. Los modificadores old y new le permiten acceder a los valores de las columnas antes y después del evento del trigger.


**Ejemplo 1 payment_date**

El siguiente trigger coloca la fecha de pago justo antes que se inserte valores en la tabla payment. Se utiliza la funcion NOW() para colocar la fecha.

In [1]:
use sakila;

Query OK

In [5]:
CREATE TRIGGER payment_date BEFORE INSERT ON payment
	FOR EACH ROW SET NEW.payment_date = NOW();


Trigger 'sakila.payment_date' already exists
(B(B


In [7]:
select count(*) from payment;

count(*)
16049


In [8]:
SHOW COLUMNS FROM payment;

Field,Type,Null,Key,Default,Extra
payment_id,smallint(5) unsigned,NO,PRI,NULL,auto_increment
customer_id,smallint(5) unsigned,NO,MUL,NULL,
staff_id,tinyint(3) unsigned,NO,MUL,NULL,
rental_id,int(11),YES,MUL,NULL,
amount,"decimal(5,2)",NO,,NULL,
payment_date,datetime,NO,,NULL,
last_update,timestamp,NO,,current_timestamp(),on update current_timestamp()


Se inserta un registro en *payment*

In [9]:
INSERT INTO payment (customer_id, staff_id,rental_id,amount) VALUES (1,1,76,2.99);

Query OK

In [10]:
select count(*) from payment;

count(*)
16050


Se selecciona el ultimo registro de *payment* donde se puede apreciar la fecha que genero el trigger. 

In [12]:
SELECT * FROM payment ORDER BY payment_id DESC LIMIT 1;

payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
16050,1,1,76,2.99,2021-05-04 22:16:16,2021-05-04 22:16:16


**Ejemplo 2 ins_film** 

El siguiente trigger genera un registro en la tabla film_text luego que se inserta una fila en la tabla film. 


A continuacion se crea el trigger ( que ya està en la base )

In [2]:
DELIMITER ;;
CREATE TRIGGER `ins_film` AFTER INSERT ON `film` FOR EACH ROW BEGIN
    INSERT INTO film_text (film_id, title, description)
        VALUES (new.film_id, new.title, new.description);
  END;;


Trigger 'sakila.ins_film' already exists
(B(B


A continuaciòn se inserta una fila en la tabla *film* 

In [9]:
INSERT INTO film (title, description,release_year,language_id,length,rating,
                  special_features)
                  
VALUES ('La vendedora de rosas','Una pelicula de drogas hecha en medellin',2000,1,86,'PG','Deleted Scenes,Behind the Scenes');

Query OK

Se muestra el ultimo registro que se ingresò a la tabla film

In [13]:
SELECT * FROM film  ORDER BY film_id DESC LIMIT 1;

film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
1001,La vendedora de rosas,Una pelicula de drogas hecha en medellin,2000,1,NULL,3,4.99,86,19.99,PG,"Deleted Scenes,Behind the Scenes",2021-05-08 14:10:30


Se activò el trigger que inserta un registro en la tabla *film_text* 

In [14]:
SELECT * FROM film_text  ORDER BY film_id DESC LIMIT 1;

film_id,title,description
1001,La vendedora de rosas,Una pelicula de drogas hecha en medellin


**Funciones**

* ¿Que es una funcion?: Es un programa almacenado en el que se ingresan parametros y luego devuelve un valor
* La estructura para crear una funciòn es: 

    CREATE 
[ DEFINER = { CURRENT_USER | user_name } ] 
FUNCTION function_name [ (parameter datatype [, parameter datatype]) ]
RETURNS return_datatype [ LANGUAGE SQL
                        | DETERMINISTIC
                        | NOT DETERMINISTIC
                        | { CONTAINS SQL 
                          | NO SQL
                          | READS SQL DATA
                          | MODIFIES SQL DATA }
                        | SQL SECURITY { DEFINER | INVOKER }
                        | COMMENT 'comment_value'

BEGIN

   declaration_section

   executable_section

END;

En la clausula definer se coloca el usuario que ha creado la funciòn.
En los tipos de datos que puede retornar se encuentran:

* Lenguaje sql.
* Deterministico.
* Contenido SQL.
* Lectura de datos sql.


Ejemplo 1: 

La funcion *The inventory_held_by_customer* devuelve el *customer_id* del cliente que ha alquilado el artículo de inventario especificado.

Esta función devuelve el customer_id del cliente que actualmente está alquilando el artículo, o NULL si el artículo está en stock. 

In [19]:
DELIMITER ;
DELIMITER $$

CREATE FUNCTION inventory_held_by_customer(p_inventory_id INT) RETURNS INT
READS SQL DATA
BEGIN
  DECLARE v_customer_id INT;
  DECLARE EXIT HANDLER FOR NOT FOUND RETURN NULL;

  SELECT customer_id INTO v_customer_id
  FROM rental
  WHERE return_date IS NULL
  AND inventory_id = p_inventory_id;

  RETURN v_customer_id;
END $$;

FUNCTION inventory_held_by_customer already exists
ERROR 1065 (42000) at line 4 in file: '/home/camilo/Descargas/.mariadb_statement': Query was empty
(B(B(B(B


Veamos por ejemplo el id del cliente que actualmente esta alquilando el item 8

In [21]:
SELECT inventory_held_by_customer(8);

inventory_held_by_customer(8)
NULL


Nos dio un valor Null ya que el item 8 actualmente esta en stock.

In [25]:
SELECT * 
  FROM rental
  where inventory_id = 8;

rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
10141,2005-07-31 22:08:29,8,8,2005-08-06 16:59:29,1,2006-02-15 21:30:53
12651,2005-08-18 18:36:16,8,34,2005-08-22 22:01:16,1,2006-02-15 21:30:53


In [27]:
SELECT inventory_held_by_customer(9);

inventory_held_by_customer(9)
366


Entonces el id del cliente que tiene actualmente el item 9 es 366

In [28]:
SELECT * 
  FROM rental
  where inventory_id = 9;

rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
10310,2005-08-01 04:24:47,9,271,2005-08-04 05:36:47,2,2006-02-15 21:30:53
13421,2006-02-14 15:16:03,9,366,NULL,1,2006-02-15 21:30:53


**Procedimientos**



Ejemplo 1

Procedimiento *The film_in_stock Stored*

Este procedimiento determina si hay copias de una película determinada en stock en una tienda determinada.



A continuacion se crea el procedimiento. 

In [29]:
DELIMITER ;
DELIMITER $$

CREATE PROCEDURE film_in_stock(IN p_film_id INT, IN p_store_id INT, OUT p_film_count INT)
READS SQL DATA
BEGIN
     SELECT inventory_id
     FROM inventory
     WHERE film_id = p_film_id
     AND store_id = p_store_id
     AND inventory_in_stock(inventory_id);

     SELECT COUNT(*)
     FROM inventory
     WHERE film_id = p_film_id
     AND store_id = p_store_id
     AND inventory_in_stock(inventory_id)
     INTO p_film_count;
END $$;

PROCEDURE film_in_stock already exists
ERROR 1065 (42000) at line 4 in file: '/home/camilo/Descargas/.mariadb_statement': Query was empty
(B(B(B(B


Ahora vamos a determinar si el film con id igual a 1 tiene copias en la tienda con id 1 y se almacenará la cantidad de copias en la variable @count

In [30]:
CALL film_in_stock(1,1,@count);

inventory_id
1
2
3
4


In [31]:
SELECT @count;

@count
4


Ahora verifiquemos la cantidad de copias del film con id=1  en la tienda con id = 1.

In [32]:
select * from inventory
WHERE film_id = 1
AND store_id = 1;

inventory_id,film_id,store_id,last_update
1,1,1,2006-02-15 05:09:17
2,1,1,2006-02-15 05:09:17
3,1,1,2006-02-15 05:09:17
4,1,1,2006-02-15 05:09:17
